<a href="https://colab.research.google.com/github/JamesGilBritodeSousa/Linux/blob/main/MongodBUbuntu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Como instalar o MongoDB no Ubuntu 20.04**

Uma versão anterior deste tutorial foi escrita por Melissa Anderson e Mark Drake
Published on September 11, 2020 7.1

### Introdução
O MongoDB, também conhecido como Mongo, é um banco de dados de documentos de código aberto usado em muitas aplicações Web modernas. Ele é classificado como um banco de dados NoSQL porque ele não depende de uma estrutura de banco de dados relacional tradicional baseada em tabelas.

Em vez disso, ele usa documentos do tipo JSON com esquemas dinâmicos, o que significa que, ao contrário de bancos de dados relacionais, o MongoDB não requer um esquema predefinido antes de adicionar dados a um banco de dados. Você pode alterar o esquema a qualquer momento e tantas vezes quanto for necessário sem precisar configurar um banco de dados novo com um esquema atualizado.

Neste tutorial, você instalará o MongoDB em um servidor Ubuntu 20.04, irá testá-lo e aprender como gerenciá-lo como um serviço systemd.

### Pré-requisitos
Para seguir este tutorial, será necessário:
Um servidor Ubuntu 20.04. Cada servidor deverá ter um usuário administrativo não root e um firewall configurado com o UFW.


## Passo 1 — Como instalar o MongoDB
Os repositórios de pacotes oficiais do Ubuntu incluem uma versão estável do MongoDB. No entanto, no momento da escrita deste artigo, a versão do MongoDB disponível a partir dos repositórios padrão do Ubuntu é a 3.6, enquanto a versão estável mais recente é a 4.4.

Para obter a versão mais recente deste software, você deve incluir o repositório de pacotes dedicado do MongoDB nas suas fontes APT. Assim, você será capaz de instalar o mongodb-org, um meta-pacote que sempre aponta para a versão mais recente do MongoDB.

Para começar, importe a chave GPG pública para a versão estável mais recente do MongoDB. Você pode encontrar o arquivo de chave apropriado, navegando até o servidor de chaves do MongoDB e encontrando o arquivo que inclui o número de versão estável mais recente e que termina em .asc. Por exemplo, se você quiser instalar a versão 4.4 do MongoDB, você procuraria por um arquivo chamado server-4.4.asc.

Clique com o botão direito no arquivo, e selecione Copy link address. Em seguida, cole esse link no comando curl a seguir, substituindo a URL destacada:

In [ ]:
!curl -fsSL https://www.mongodb.org/static/pgp/server-4.4.asc | sudo apt-key add -


OK


O cURL é uma ferramenta de linha de comando disponível em muitos sistemas operacionais, usada para transferir dados. Ele lê todos os dados armazenados na URL passada a ele e imprime o conteúdo na saída do sistema. No exemplo a seguir, o cURL imprime o conteúdo do arquivo de chave GPG e então faz um pipe para o seguinte comando sudo apt-key add -, adicionando assim a chave GPG à sua lista de chaves confiáveis.

Além disso, note que este comando curl usa as opções -fsSL que, juntas, essencialmente dizem ao cURL para falhar silenciosamente. Isso significa que, se por alguma razão o cURL não for capaz de entrar em contato com o servidor GPG ou o servidor GPG estiver fora do ar, ele não adicionará acidentalmente o código de erro resultante à sua lista de chaves confiáveis.

Este comando retornará OK se a chave for adicionada com sucesso:

Output
OK

Se quiser verificar se a chave foi adicionada corretamente, você pode fazer isso com o seguinte comando:


In [ ]:
!apt-key list

'''Isso irá retornar a chave do MongoDB em algum lugar na saída:

Output
/etc/apt/trusted.gpg
--------------------
pub   rsa4096 2019-05-28 [SC] [expires: 2024-05-26]
      2069 1EEC 3521 6C63 CAF6  6CE1 6564 08E3 90CF B1F5
uid           [ unknown] MongoDB 4.4 Release Signing Key <packaging@mongodb.com>
. . .'''

Neste ponto, sua instalação APT ainda não encontrará o pacote mongodb-org que você precisa para instalar a versão mais recente do MongoDB.

Há dois lugares no servidor onde o APT procura fontes online de pacotes para baixar e instalar: o arquivo sources.list e o diretório sources.list.d. O sources.list é um arquivo que lista fontes ativas de dados APT, com uma fonte por linha e as fontes mais preferidas listadas primeiro. O diretório sources.list.d permite que você adicione essas entradas sources.list como arquivos separados.

Execute o comando a seguir, que cria um arquivo no diretório sources.list.d chamado mongodb-org-4.4.list. O único conteúdo neste arquivo é uma linha contendo deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu focal/mongodb-org/4.4 multiverse:

In [ ]:

!echo "deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu focal/mongodb-org/4.4 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-4.4.list

Esta única linha diz ao APT tudo o que ele precisa saber sobre o que é a fonte e onde encontrá-la:

deb: isso significa que a entrada da fonte faz referência a uma arquitetura Debian regular. Em outros casos, essa parte da linha 

*   List item
*   List item

pode aparecer como deb-src, o que significa que a entrada da fonte representa o código fonte da distribuição Debian.

[ arch=amd64,arm64 ]: isso especifica para quais arquiteturas os dados APT devem ser baixados. Neste caso, ele especifica as arquiteturas amd64 e arm64.

https://repo.mongodb.org/apt/ubuntu: este é um URI representando a localização onde os dados APT podem ser encontrados. Neste caso, o URI aponta para o endereço HTTPS onde o repositório oficial do MongoDB está localizado.

focal/mongodb-org/4.4: os repositórios do Ubuntu podem conter várias versões diferentes. Isso especifica que você só quer a versão 4.4 do pacote mongodb-org disponível para a versão focal do Ubuntu (“Focal Fossa” sendo o codinome do Ubuntu 20.04).

multiverse: essa parte aponta o APT para um dos quatro principais repositórios do Ubuntu. Neste caso, ele está apontando para o repositório multiverse.

In [ ]:
# Depois de executar este comando, atualize o índice local de pacotes do servidor para que o APT possa encontrar o pacote mongodb-org:
!sudo apt update

In [ ]:
# Depois disso, instale o MongoDB:
!sudo apt install mongodb-org

Quando solicitado, pressione Y, e depois, ENTER para confirmar que deseja instalar o pacote:

Quando o comando terminar, o MongoDB estará instalado em seu sistema. No entanto, ainda não será possível usá-lo. Em seguida, você iniciará o MongoDB e confirmará se ele está funcionando corretamente.

## Passo 2 — Iniciando o serviço do MongoDB e testando o banco de dados
O processo de instalação descrito no passo anterior configura automaticamente o MongoDB para ser executado como um daemon controlado pelo systemd, o que significa que você pode gerenciar o MongoDB usando os vários comandos systemctl. No entanto, esse procedimento de instalação não inicia automaticamente o serviço.

Execute o comando systemctl a seguir para iniciar o serviço MongoDB:

In [ ]:
!sudo systemctl start mongod.service

Em seguida, verifique o status do serviço. Observe que esse comando não inclui .service na definição do arquivo de serviço. O systemctl irá anexar esse sufixo a qualquer argumento que você passa automaticamente se ele ainda não estiver presente. Desse modo, não é necessário incluí-lo:


In [ ]:
!sudo systemctl status mongod

'''Este comando retornará uma saída como a seguinte, indicando que o serviço está funcionando:

Output
● mongod.service - MongoDB Database Server
     Loaded: loaded (/lib/systemd/system/mongod.service; disabled; vendor preset: enabled)
     Active: active (running) since Tue 2020-06-09 12:57:06 UTC; 2s ago
       Docs: https://docs.mongodb.org/manual
   Main PID: 37128 (mongod)
     Memory: 64.8M
     CGroup: /system.slice/mongod.service
             └─37128 /usr/bin/mongod --config /etc/mongod.conf'''

Depois de confirmar que ele está funcionando como esperado, habilite o serviço MongoDB para iniciar durante a inicialização:

In [ ]:
!sudo systemctl enable mongod

Você pode verificar se o banco de dados está operacional conectando-se ao servidor de banco de dados e executando um comando de diagnóstico. O comando a seguir conectará você ao banco de dados e mostrará na saída a versão atual, endereço do servidor e a porta dele. Ele também retornará o resultado do comando interno do MongoDB connectionStatus:


In [ ]:
!mongo --eval 'db.runCommand({ connectionStatus: 1 })'

'''connectionStatus verificará e retornará o status da conexão com o banco de dados. Um valor de 1 para o campo ok na resposta indica que o servidor está funcionando corretamente.

Output
MongoDB shell version v4.4.0
connecting to: mongodb://127.0.0.1:27017/?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("1dc7d67a-0af5-4394-b9c4-8a6db3ff7e64") }
MongoDB server version: 4.4.0
{
    "authInfo" : {
        "authenticatedUsers" : [ ],
        "authenticatedUserRoles" : [ ]
    },
    "ok" : 1
}'''

Além disso, note que o banco de dados é executado na porta 27017 em 127.0.0.1, o endereço de loopback local representando o localhost. Este é o número padrão da porta do MongoDB.

Em seguida, analisaremos como gerenciar a instância do servidor MongoDB com o systemd.

## Passo 3 — Como gerenciar o serviço MongoDB
Como mencionado anteriormente, o processo de instalação descrito no Passo 1 configura o MongoDB para ser executado como um serviço systemd. Isso significa que você pode gerenciá-lo usando comandos padrão systemctl como faria com outros serviços do sistema Ubuntu.


In [ ]:
# Como mencionado anteriormente, o comando systemctl status verifica o status do serviço MongoDB:
!sudo systemctl status mongod

In [ ]:
# Você pode parar o serviço a qualquer momento digitando:
!sudo systemctl stop mongod

In [ ]:
# Para iniciar o serviço quando ele estiver parado, execute:
!sudo systemctl start mongod

In [ ]:
# Você também pode reiniciar o servidor quando ele já estiver executando:
!sudo systemctl restart mongod

In [ ]:
# Se você quiser desativar essa inicialização automática, digite:
!sudo systemctl disable mongod

In [ ]:
# Em seguida, para habilitá-lo novamente durante a inicialização, execute o comando enable novamente:
!sudo systemctl enable mongod

In [ ]:
# Se necessário desativar o firewall:
!sudo ufw disable

!sudo ufw status